In [107]:
import torch
import torch.nn as nn
import torch.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

In [108]:
class NeuronalNet(nn.Module):
        def __init__(self):
            super(NeuronalNet, self).__init__()
            self.lin1 = nn.Linear(10,10)
            self.lin2 = nn.Linear(10,10)
            self.lin3 = nn.Linear(10,10)
        def forward(self,x):
            x = self.lin1(x)
            x = F.relu(x)
            x = F.relu(self.lin2(x))
            x = F.relu(self.lin3(x))
            return x

In [109]:
net = NeuronalNet()

In [110]:
england = pd.read_csv('/Users/fabian-maltemoller/Developer/Ki/KI_Daten/TENYEARS.csv', sep=';')

In [111]:
df = pd.DataFrame(england, columns = ['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR'])

In [112]:
print(df)

          Date     HomeTeam      AwayTeam  FTHG  FTAG FTR
0     15.08.09  Aston Villa         Wigan     0     2   A
1     15.08.09    Blackburn      Man City     0     2   A
2     15.08.09       Bolton    Sunderland     0     1   A
3     15.08.09      Chelsea          Hull     2     1   H
4     15.08.09      Everton       Arsenal     1     6   A
...        ...          ...           ...   ...   ...  ..
3795  12.05.19    Liverpool        Wolves     2     0   H
3796  12.05.19   Man United       Cardiff     0     2   A
3797  12.05.19  Southampton  Huddersfield     1     1   D
3798  12.05.19    Tottenham       Everton     2     2   D
3799  12.05.19      Watford      West Ham     1     4   A

[3800 rows x 6 columns]


In [113]:
dfAstonVilla = df.loc[df['HomeTeam'] == "Aston Villa"]

In [114]:
dfAstonVilla.loc[dfAstonVilla['AwayTeam'] == "Tottenham"]

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
129,28.11.09,Aston Villa,Tottenham,1,1,D
552,26.12.10,Aston Villa,Tottenham,1,2,A
1121,06.05.12,Aston Villa,Tottenham,1,1,D
1319,26.12.12,Aston Villa,Tottenham,0,4,A
1598,20.10.13,Aston Villa,Tottenham,0,2,A
1997,02.11.14,Aston Villa,Tottenham,1,2,A
2571,13.03.16,Aston Villa,Tottenham,0,2,A
